In [1]:
import warnings
warnings.filterwarnings('ignore')


import json
import pandas as pd
import numpy as np
import os
import glob
import re
from pathlib import Path

import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from collections import Counter
import matplotlib as plt
from sqlalchemy import create_engine
import psycopg2
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
# from config import db_password
from imblearn.under_sampling import ClusterCentroids 
import time
from imblearn.combine import SMOTEENN
# import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [2]:
file_path = Path("/content/tmintest.csv")

In [3]:
df = pd.read_csv(file_path,index_col = 0)
df.head()

,trip_id,depart_airport,return_airport,depart_date,return_date,total_cost,depart_datetime,depart_duration,depart_stops,return_datetime,return_duration,return_stops,day_before_purchase,total_min
timestamp,,,,,,,,,,,,,,
2022-06-23 03:41:00,SNA-SMF-06/23/2022-06/26/2022,SNA,SMF,2022-06-23,6/26/2022,720,6/23/2022 10:17,309,1,6/26/2022 6:00,341,1,0,0
2022-06-23 03:42:00,SNA-SMF-06/30/2022-07/03/2022,SNA,SMF,2022-06-30,7/3/2022,630,6/30/2022 10:17,309,1,6/30/2022 10:17,309,1,6,6
2022-06-23 03:43:00,SNA-PHX-06/23/2022-06/26/2022,SNA,PHX,2022-06-23,6/26/2022,353,6/23/2022 12:56,78,0,6/26/2022 10:13,88,0,0,0
2022-06-23 03:43:00,SNA-PHX-06/30/2022-07/03/2022,SNA,PHX,2022-06-30,7/3/2022,299,6/30/2022 12:56,78,0,7/3/2022 14:59,78,0,6,6
2022-06-23 03:44:00,SNA-ORD-06/23/2022-06/26/2022,SNA,ORD,2022-06-23,6/26/2022,2939,6/23/2022 16:44,636,1,6/26/2022 8:25,251,0,0,0


In [4]:
df = df[['depart_airport','return_airport','total_cost','depart_duration','depart_stops','return_duration','return_stops','day_before_purchase','total_min']]

In [5]:
df.head()

,depart_airport,return_airport,total_cost,depart_duration,depart_stops,return_duration,return_stops,day_before_purchase,total_min
timestamp,,,,,,,,,
2022-06-23 03:41:00,SNA,SMF,720,309,1,341,1,0,0
2022-06-23 03:42:00,SNA,SMF,630,309,1,309,1,6,6
2022-06-23 03:43:00,SNA,PHX,353,78,0,88,0,0,0
2022-06-23 03:43:00,SNA,PHX,299,78,0,78,0,6,6
2022-06-23 03:44:00,SNA,ORD,2939,636,1,251,0,0,0


In [6]:
df.dtypes

depart_airport         object
return_airport         object
total_cost              int64
depart_duration         int64
depart_stops            int64
return_duration         int64
return_stops            int64
day_before_purchase     int64
total_min               int64
dtype: object

In [7]:
print(df['depart_airport'].unique())

['SNA' 'LAX' 'SFO' 'SJC']


In [8]:
print(df['return_airport'].unique())

['SMF' 'PHX' 'ORD' 'IAD' 'MSY']


In [9]:
df['depart_airport'].value_counts()

SNA    20
LAX    15
SFO     5
SJC     5
Name: depart_airport, dtype: int64

In [10]:
df['return_airport'].value_counts()

ORD    17
PHX    16
IAD     5
MSY     5
SMF     2
Name: return_airport, dtype: int64

In [11]:
df.shape

(45, 9)

In [12]:
columns = ["depart_airport","return_airport",
           "total_cost","depart_duration",
           "depart_stops","return_duration","return_stops",
           "day_before_purchase","total_min"
    
]

# target = ["total_cost"]
target = ["day_before_purchase"]

In [13]:
# Create our features
# X = df2.drop(columns='total_cost')
X = df[["day_before_purchase", "return_duration", "return_stops", "depart_duration", "depart_airport", "return_airport"]] #, "depart_airport", "return_airport"
X = pd.get_dummies(X)

# Create our target
y = df.loc[:, target].copy()

In [14]:
X.head()

,day_before_purchase,return_duration,return_stops,depart_duration,depart_airport_LAX,depart_airport_SFO,depart_airport_SJC,depart_airport_SNA,return_airport_IAD,return_airport_MSY,return_airport_ORD,return_airport_PHX,return_airport_SMF
timestamp,,,,,,,,,,,,,
2022-06-23 03:41:00,0,341,1,309,0,0,0,1,0,0,0,0,1
2022-06-23 03:42:00,6,309,1,309,0,0,0,1,0,0,0,0,1
2022-06-23 03:43:00,0,88,0,78,0,0,0,1,0,0,0,1,0
2022-06-23 03:43:00,6,78,0,78,0,0,0,1,0,0,0,1,0
2022-06-23 03:44:00,0,251,0,636,0,0,0,1,0,0,1,0,0


In [15]:
# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [58]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

In [59]:
# Add the first input layer with relu
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=13))

In [60]:
nn_model.add(tf.keras.layers.Dense(units=16, activation="exponential"))

In [61]:
nn_model.add(tf.keras.layers.Dense(units=8, activation="tanh"))

In [62]:
# nn_model.add(tf.keras.layers.Dense(units=4, activation="softmax"))

In [63]:
# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [64]:
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 32)                448       
                                                                 
 dense_19 (Dense)            (None, 8)                 264       
                                                                 
 dense_20 (Dense)            (None, 1)                 9         
                                                                 
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [66]:
# tf.keras.models.Model.build build(input_shape)

In [67]:
fit_mdl = nn_model.fit(X_train, y_train, epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 8ms/step - loss: 110.8045 - accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 0s 8ms/step - loss: 110.3192 - accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 110.0399 - accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 0s 6ms/step - loss: 109.8510 - accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 0s 5ms/step - loss: 109.7261 - accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 0s 5ms/step - loss: 109.5912 - accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 0s 5ms/step - loss: 109.4918 - accuracy: 0.0000e+00
Epoch 8/10
2/2 [==============================] - 0s 12ms/step - loss: 109.4599 - accuracy: 0.0000e+00
Epoch 9/10
2/2 [==============================] - 0s 7ms/step - loss: 109.4320 - accuracy: 0.0000e+00
Epoch 10/10
2/2 [==============================] - 0s 7ms/step - loss: 109.4053 -

In [69]:
nn_model.predict(X_test)

array([[0.8351561 ],
       [0.7217762 ],
       [0.8351332 ],
       [0.8351561 ],
       [0.8350704 ],
       [0.8351561 ],
       [0.8351509 ],
       [0.8351399 ],
       [0.8351561 ],
       [0.72177017],
       [0.8351561 ],
       [0.8351561 ]], dtype=float32)

In [70]:
y_test

,day_before_purchase
timestamp,
2022-06-23 04:48:32,27
2022-06-23 04:54:01,0
2022-06-23 04:52:12,20
2022-06-24 01:51:01,12
2022-06-23 04:41:02,6
2022-06-23 04:43:34,27
2022-06-25 00:00:00,12
2022-06-23 04:51:19,13
2022-06-24 01:52:03,5
